I. Tiền xử lý và chuẩn bị dữ liệu

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

file_path = r'D:\project\visual\data\owid-co2-data.csv'

df = pd.read_csv(file_path)



C:\Users\vunambac\AppData\Local\Temp\ipykernel_7216\775061852.py:8: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
print("Kích thước dữ liệu:", df.shape)
print("\nThông tin cột:")
print(df.info())
print("\nMẫu dữ liệu đầu tiên:")
print(df.head())

missing_values = df.isnull().sum()
print("\nGiá trị thiếu theo cột:")
print(missing_values[missing_values > 0])

In [ ]:
print("\nKiểu dữ liệu từng cột:")
print(df.dtypes)

# Tìm các cột dạng object có thể chứa giá trị sai kiểu (ví dụ: số nhưng lưu dạng chuỗi)
for col in df.select_dtypes(include='object').columns:
    print(f"\nCác giá trị khác thường trong cột {col}:")
    print(df[col].value_counts().head(10))


In [ ]:
# Xem tất cả các giá trị khác nhau trong cột 'country'
unique_countries = df['country'].unique()
print("Số lượng quốc gia khác nhau:", len(unique_countries))
print("\nDanh sách các quốc gia:")
print(unique_countries)


In [3]:
duplicate_rows = df[df.duplicated()]
print("\nSố dòng trùng lặp:", duplicate_rows.shape[0])



Số dòng trùng lặp: 0


In [ ]:
print("\nThống kê mô tả các cột số:")
print(df.describe())

# Trực quan hoá nhanh
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols].hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()


In [ ]:
cols_to_check = ['co2', 'gdp', 'population']
for col in cols_to_check:
    if col in df.columns:
        print(f"\nGiá trị âm hoặc 0 trong cột {col}:")
        print(df[df[col] <= 0][['country', 'year', col]].head())


In [ ]:
print("\nSố năm dữ liệu theo quốc gia:")
print(df.groupby('country')['year'].nunique().sort_values(ascending=False).head())

# Kiểm tra missing theo quốc gia
missing_by_country = df.groupby('country').apply(lambda x: x.isnull().sum())
print("\nGiá trị thiếu theo quốc gia (top 5):")
print(missing_by_country.head())


In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm', annot=False)
plt.title("Ma trận tương quan giữa các biến số")
plt.show()


In [11]:
print("\nKhoảng thời gian dữ liệu:")
print("Từ năm:", df['year'].min(), "→ đến năm:", df['year'].max())



Khoảng thời gian dữ liệu:
Từ năm: 1750 → đến năm: 2023


II. Phân tích phát thải toàn cầu

1.Line Chart: Xu hướng phát thải toàn cầu (1960–2023)

In [4]:
import plotly.express as px
# Lọc dữ liệu toàn cầu
world = df[df['country'] == 'World']
world = world[(world['year'] >= 1750) & (world['year'] <= 2023)]

# Vẽ biểu đồ tương tác
fig = px.line(
    world,
    x='year',
    y='co2',
    title='Tổng phát thải CO₂ toàn cầu (1750–2023)',
    labels={'year': 'Năm', 'co2': 'Phát thải CO₂ (triệu tấn)'},
    hover_data={'year': True, 'co2': ':.2f'},  # hiển thị 2 chữ số thập phân
    template='plotly_dark'  # hoặc 'plotly_white' nếu muốn nền sáng
)

# Tùy chỉnh hiển thị
fig.update_traces(line=dict(color='red', width=3))
fig.update_layout(
    hovermode='x unified',
    font=dict(size=14),
    title_x=0.5
)

# Hiển thị biểu đồ
fig.show()

2.Choropleth Map: Phát thải theo quốc gia

In [36]:

# Giữ dữ liệu cấp quốc gia
countries = df[df['iso_code'].apply(lambda x: isinstance(x, str) and len(x) == 3)]
countries = countries[(countries['year'] >= 1950) & (countries['year'] <= 2023)]
countries = countries.dropna(subset=['co2', 'iso_code'])

# Xác định khoảng giá trị toàn bộ (để thang màu không đổi theo năm)
vmin = countries['co2'].min()
vmax = countries['co2'].max()

# Vẽ bản đồ
fig = px.choropleth(
    countries,
    locations='iso_code',
    color='co2',
    hover_name='country',
    animation_frame='year',
    color_continuous_scale='RdYlBu_r',   
    range_color=[vmin, vmax],           
    #title='Phát thải CO₂ theo quốc gia (1950–2023)',
    labels={'co2': 'Phát thải CO₂ (triệu tấn)', 'year': 'Năm'}
)

# Tùy chỉnh thanh màu và bố cục
fig.update_layout(
    title_x=0.5,
    font=dict(size=12),
    hovermode='closest',
    coloraxis_colorbar=dict(
        title='CO₂ (triệu tấn)',
        tickformat='.0f',
        ticks='outside',
        lenmode='fraction',
        len=0.9,
        yanchor='middle',
        y=0.5,
        outlinewidth=1,
        outlinecolor='black'
    )
)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100   # mỗi frame 0.25 giây
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 200

# Hiển thị
fig.show()


3.Bar Chart: Top 10 quốc gia phát thải cao nhất

In [37]:
import pandas as pd
import plotly.express as px

# Giữ dữ liệu cấp quốc gia
countries = df[df['iso_code'].apply(lambda x: isinstance(x, str) and len(x) == 3)]
countries = countries[(countries['year'] >= 1950) & (countries['year'] <= 2023)]
countries = countries.dropna(subset=['co2', 'iso_code'])

# Tính tổng phát thải CO₂ của mỗi quốc gia trong giai đoạn 1950–2023
top_emitters = (
    countries.groupby('country', as_index=False)['co2']
    .sum()
    .sort_values(by='co2', ascending=False)
    .head(10)
)

# Vẽ biểu đồ cột
fig = px.bar(
    top_emitters,
    x='country',
    y='co2',
    color='co2',
    color_continuous_scale='RdYlBu_r',  # Đỏ → xanh (phân biệt mạnh)
    labels={'co2': 'Tổng phát thải CO₂ (triệu tấn)', 'country': 'Quốc gia'},
    title='Top 10 quốc gia phát thải CO₂ cao nhất (1950–2023)'
)

# Tùy chỉnh hiển thị
fig.update_layout(
    title_x=0.5,
    xaxis_tickangle=-30,
    font=dict(size=13),
    coloraxis_colorbar=dict(title='CO₂ (triệu tấn)')
)

fig.show()


Sự thay đổi của các nguồn phát thải CO₂ toàn cầu (1750–2023)

In [5]:


# Lọc dữ liệu toàn cầu
world = df[df['country'] == 'World']
world = world[(world['year'] >= 1750) & (world['year'] <= 2023)]

# Chọn các nguồn phát thải chính
sources = ['coal_co2', 'oil_co2', 'gas_co2', 'cement_co2', 'flaring_co2', 'land_use_change_co2']

# Chuyển dữ liệu từ dạng wide → long để vẽ stacked area
world_long = world.melt(
    id_vars=['year'],
    value_vars=sources,
    var_name='Nguồn phát thải',
    value_name='Lượng CO₂ (triệu tấn)'
)

# Đổi tên các nguồn cho dễ đọc
name_map = {
    'coal_co2': 'Than đá',
    'oil_co2': 'Dầu mỏ',
    'gas_co2': 'Khí đốt tự nhiên',
    'cement_co2': 'Xi măng',
    'flaring_co2': 'Đốt khí thừa',
    'land_use_change_co2': 'Thay đổi sử dụng đất'
}
world_long['Nguồn phát thải'] = world_long['Nguồn phát thải'].map(name_map)

# Vẽ biểu đồ stacked area chart tương tác
fig = px.area(
    world_long,
    x='year',
    y='Lượng CO₂ (triệu tấn)',
    color='Nguồn phát thải',
    title='Các nguồn phát thải CO₂ toàn cầu (1750–2023)',
    labels={'year': 'Năm', 'Lượng CO₂ (triệu tấn)': 'Phát thải CO₂ (triệu tấn)'},
    template='plotly_white'
)

# Tùy chỉnh giao diện
fig.update_layout(
    hovermode='x unified',
    title_x=0.5,
    font=dict(size=14),
    legend_title_text='Nguồn phát thải',
    yaxis=dict(title='Phát thải CO₂ (triệu tấn)')
)

# Hiển thị
fig.show()


III. Phân tích phát thải Việt Nam

1.Line Chart: Xu hướng phát thải CO₂ tại Việt Nam

In [41]:
import plotly.express as px

# Lọc dữ liệu Việt Nam
vietnam = df[df['country'] == 'Vietnam']
vietnam = vietnam[(vietnam['year'] >= 1750) & (vietnam['year'] <= 2023)]
vietnam = vietnam.dropna(subset=['co2'])

# Vẽ biểu đồ tương tác
fig = px.line(
    vietnam,
    x='year',
    y='co2',
    title='Phát thải CO₂ của Việt Nam (1750–2023)',
    labels={'year': 'Năm', 'co2': 'Phát thải CO₂ (triệu tấn)'},
    hover_data={'year': True, 'co2': ':.2f'},
    template='plotly_white'  # hoặc 'plotly_dark' nếu thích nền tối
)

# Tùy chỉnh hiển thị
fig.update_traces(line=dict(color='royalblue', width=3))
fig.update_layout(
    hovermode='x unified',
    font=dict(size=14),
    title_x=0.5,
    plot_bgcolor='rgba(240,240,240,0.9)',  # nền nhẹ
    margin=dict(l=60, r=60, t=60, b=60)
)

# Hiển thị
fig.show()


2.Stacked Area Chart: Phân tách phát thải theo ngành

In [42]:
import plotly.express as px
import pandas as pd

# Lọc dữ liệu Việt Nam
vn = df[df['country'] == 'Vietnam']
vn = vn[(vn['year'] >= 1950) & (vn['year'] <= 2023)]

# Chọn các ngành chính
vn = vn[['year', 'coal_co2', 'oil_co2', 'gas_co2', 'cement_co2', 'land_use_change_co2']]
vn = vn.dropna(how='any')

# Đổi tên cột để hiển thị đẹp hơn
vn = vn.rename(columns={
    'coal_co2': 'Than',
    'oil_co2': 'Dầu',
    'gas_co2': 'Khí đốt',
    'cement_co2': 'Xi măng',
    'land_use_change_co2': 'Thay đổi sử dụng đất'
})

# Chuyển dữ liệu sang dạng "long" để plotly hiểu
vn_melt = vn.melt(id_vars='year', var_name='Ngành', value_name='Phát thải (triệu tấn)')

# Vẽ stacked area chart
fig = px.area(
    vn_melt,
    x='year',
    y='Phát thải (triệu tấn)',
    color='Ngành',
    title='Cơ cấu phát thải CO₂ theo ngành tại Việt Nam (1950–2023)',
    labels={'year': 'Năm'},
    template='plotly_white'
)

# Tùy chỉnh hiển thị
fig.update_layout(
    hovermode='x unified',
    font=dict(size=14),
    title_x=0.5,
    legend_title_text='Nguồn phát thải',
    yaxis_title='Phát thải CO₂ (triệu tấn)',
    xaxis_title='Năm',
    plot_bgcolor='rgba(240,240,240,0.9)',
    margin=dict(l=60, r=60, t=60, b=60)
)

# Hiển thị
fig.show()


IV. Dự báo và mô hình học máy

1. Chuẩn bị dữ liệu chuỗi thời gian

2. Mô hình dự báo

3. Biểu đồ dự báo

V. Dự báo nhiệt độ và nhu cầu năng lượng

1.Multiple Regression: Temperature ~ CO2 + Energy

2.Biểu đồ nhiệt độ dự báo đến năm 2050

3.Tương quan đa chiều (Heatmap)